# What Makes A Best Original Screenplay?

    By Allison Ragan and Allison Shafer
    American University
    STAT-696 Applied Natural Language Processing

# Objective

# Table of Contents

    I. Package Imports
    II. NLP Transformations
        i. CountVectorized
        ii. TFIDF
    III. Sentiment Analysis
        i. Sentiment Analysis -- Dialogue
        ii. Sentiment Analysis -- Full Script
    IV. Topic Modeling
        i. Topic Modeling -- Dialogue
        ii. Topic Modeling -- Full Script
    V. K-Means Clustering
        i. K-Means Clustering -- Dialogue
        ii. K-Means Clustering -- Full Script
    VI. Results
    VII. Deprecated Code

## Package Imports

In [4]:
import pandas as pd 
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
import seaborn as sns

In [7]:
# Read in .csv to dataframe

movies_df = pd.read_csv('./movies_df.csv')

## NLP transformations

### CV

In [ ]:
# instantiate
bow_all = CountVectorizer(ngram_range=(1,2))
# fit + transform training data
bow_all_fit = bow_all.fit_transform(movies_df['cleaned_script'])

### TFIDF

In [ ]:
# instantiate
tfidf_all = TfidfVectorizer(max_features=5000)
# fit + transform training data
tfidf_all_fit = tfidf_all.fit_transform(movies_df['cleaned_script'])
tfidf_vocab_all = tfidf_dialogue.vocabulary_
tfidf_vocab_all = {k: v for k, v in sorted(tfidf_vocab_all.items(), key=lambda item: item[1], reverse=True)}
print(tfidf_vocab_all)b

In [ ]:
# instantiate
tfidf_dialogue = TfidfVectorizer(max_features=5000)
# fit + transform training data
tfidf_dialogue_fit = tfidf_dialogue.fit_transform(movies_df['cleaned_dialogue'])
tfidf_vocab_dialogue = tfidf_dialogue.vocabulary_
tfidf_vocab_dialogue = {k: v for k, v in sorted(tfidf_vocab_dialogue.items(), key=lambda item: item[1], reverse=True)}
print(tfidf_vocab_dialogue)

# Sentiment Analysis

maybe one person uses ANEW and one uses Harvard-IV??

or just split it up so one person does: 
- SA with dialogue CV 

and the other does:
- SA with everything CV

### Sentiment Analysis -- Dialogue

### Sentiment Analysis -- Full Script

## Topic Modeling -- AS

TFIDF

### Topic Modeling -- Dialogue

### Topic Modeling -- Full Script

## K Means Clustering  -- AR

Either on CV and TFIDF, or just on NMF

### K Means Clustering -- Dialogue

#### CV

#### TFIDF

### K Means Clustering -- Full Script

#### CV

#### TFIDF

In [ ]:
# from homework 6

# def eval_k(kmeans, centroids, terms):
#     cluster_terms = []
#     for i in range(kmeans.n_clusters):
#         termss = []
#         for ind in centroids[i, :10]:
#             termss.append(terms[ind])
#         cluster_terms.append(termss)
#     for i in range(kmeans.n_clusters):
#         print(f'\tCluster {i+1}: {" ".join(cluster_terms[i])}')

# # K-Means with a) Count Vectorized
# k_cv = KMeans(n_clusters = 20, random_state = 0).fit(cv_tf)
# k_cv_centroids = k_cv.cluster_centers_.argsort()[:, ::-1]
# k_cv_terms = cv_feat_names
# k_cv_labels = k_cv.labels_

# print("Count Vectorized:")
# eval_k(k_cv, k_cv_centroids, k_cv_terms)

# # K-Means with b) TFIDF
# k_tfidf = KMeans(n_clusters = 20, random_state = 0).fit(tfidf)
# k_tfidf_centroids = k_tfidf.cluster_centers_.argsort()[:, ::-1]
# k_tfidf_terms = cv_feat_names
# k_tfidf_labels = k_tfidf.labels_

# print("TFIDF:")
# eval_k(k_tfidf, k_tfidf_centroids, k_tfidf_terms)

# # K-Means with c) NMF -- (1,1), (1,2), (2,3) based on 4
# k_nmf = KMeans(n_clusters = 20, random_state = 0).fit(tfidf_onetwo_matrix)
# k_nmf_centroids = k_nmf.cluster_centers_.argsort()[:, ::-1]
# k_nmf_terms = tfidf_onetwo_feat_names
# k_nmf_labels = k_nmf.labels_

# print("NMF:")
# eval_k(k_nmf, k_nmf_centroids, k_nmf_terms)